# Containers e Microserviços

## Modelo em camadas MVC

__Arquitetura Monolitica__

![](mvc.png)

Todas as camadas representam uma única estrutura da aplicação, mesmo com as camadas virtuais como vemos acima!

* Conjunto de módulos numa aplicação executável
* Funções de negócios implementadas numa _única stack_
* Módulos compartilham recursos de processamento
* __Dependente__ da linguagem ou framework escolhido, não aceita mais de uma linguagem na mesma aplicação
* Maior crescimento = Maior complexidade
    * Acoplamento generalizado - " _Alta dependência de componentes de código_ "
* __Escalabilidade:__ replica o monolito inteiro
* Resiliência fragilizada
    * _Falta de memória:_ toda a aplicação é comprometida
* __Redeployment total__ a cada atualização

## Modelo baseado em serviços

__Arquitetura Microserviços__

![](micro.png)

* __Compenetização via serviços__ (multi-stack):
    * Serviços são pequenos e com deployment independentes
    * Cada serviço representa um conjunto de __regras de negócios específico__
    * Independência de tecnologia e/ou framework
    * Príncipio  "responsabilidade única" - inerente aos containers (que tb possuem)


* __Comunicação Heterogênea:__
    * HTTP, TCP, UDP, Messaging,etc..
    * __Payloads:__ JSON, BSON, XML, Protocol Buffers, etc..
    * Comunicação via APIs RESTful
    
    
* __Gerenciamento e manutenção:__
    * É maior que nos outros cenários
    * Diferentes times responsaveis por diferentes serviços
    * Atualização e substituição independentes, a atualização de um microserviço não afeta os outros microserviços da aplicação
    * Menores unidades = Maior facilidade de compreensão
    
    
* __Baixo acoplamento e Alta coesão__
* __Maior resiliência e flexibilidade__
* __Aumenta a escalabilidade__
* __Promove maior reusabilidade__
* __Persistência "poliglota"__
    * Cada serviço define seu prórpio modelo de persistência

## Visão de uma aplicação que implementa o modelo de microserviços

![](micro_serv.png)

# Rede Bridge definida pelo usuário

Permite que o usuario crie escopos personalizados

* __Scopo Local:__
    * Pode ser roteado fazendo comunicação externa
    * _driver macvlan_ é __recomendado__

* __Bridge Padrão:__
    * __NÃO__ é recomendado para uso em produção
    * A __user-defined-bridge__ é "superior" a bridge padrão

A _user-defined-bridge_ possui diversas vantagens em relação aos outros modos, dentre eles:

* Melhor isolamento e InterOperacionalidade
* Resolução automática de __DNS entre containers__, ou seja os containers passam a ser "vistos" pelos seus nomes
* Cada grupo de aplicações/serviços podem ter __sua própria rede__
* Cada container pode __conectar desconectar__ de redes definidas pelo usuário em tempo real
* Para diferentes grupos de aplicações/serviços há __diferentes requisitos__ de rede (MTU, iptables)

![](user_defined.png)

Graficamente:
* docker host é o hospedeiro
* Tendo a interface de ponte que é a interface local __ETH__
* A _rede bridge padrão_ representada pela __docker0__
    * Um container inserido nela
* A _minharede_ definida pelo usuário 
    * Os containers que farão dessa rede criada pelo usuario

## Exemplos:

### Criar a rede especificando o segmento de rede

__docker network create --subnet 10.0.0.0/24 busybox-net__

### Criando um container e atribuindo ele a rede criada

__docker run -itd --name c1 --net busybox-net busybox__


* __--net NOME_DA_REDE__ é o comando que atribui o container a rede criada

Podemos ainda explicitar o ip que queremos para um determinado container (não é muito comum)<br>

__docker run -itd --name c2 --net busybox-net --ip 10.0.0.254 -p 9090:80 nginx__

### Comunicação entre os container

Como os dois container estão na mesma rede criada eles fazem uso das vantages comentadas acima, dentre elas poderem se comunicar pelo nome: 

__docker exec -it c1 ping c2__

* c1 é container da imagem busybox que possui a option de ping

### Testando a conexão do container

__docker exec -it c1 telnet c2 80__

* _telnet_ é uma option do busybox que utiliza o protocolo telnet para teste de portas de rede
* Nesse caso estamos testando a conexão com a porta 80 do container c2

__OBS:__ Caso um container esteja em outra rede ou mesmo na rede padrão, ele não conseguirá se comunicar com os containers que estão na rede criada pelo usuario

### Case de uso

Temos uma aplicação que divide-se em __Banco__ e __App WordPress__, podemos criar uma rede docker e linkar os dois containers necessário a mesma rede, assim a comunicação passa a ser feita diretamente e ja é possível utiliza-la

* Criando a rede que será utilizada
    * __docker network create rede-wordpress__
        * _rede-wordpress_ é o nome rede (pode ser qquer um)
        

* Container do banco apartir da imagem do __mysql versão 5.7__
    * __docker run --name db-wp -e MYSQL_ROOT_PASSWORD=mysql -d --net rede-wordpress mysql:5.7__
        * Algumas imagens exigem que sejam passadas algumas variaveis de ambiente para funcionares (__-e...__)
        * O __--net rede-wordpress__ faz o link do container criado para a rede criada
    

* Container da aplicação wordpress apartir da imagem do __wordpress__
    * __docker run --name wp -e WORDPRESS_DB_HOST=db-wp -e WORDPRESS_DB_USER=root -e WORDPRESS_DB_PASSWORD=mysql -p 8080:80 -d --net rede-wordpress wordpress__
        * Novamente algumas variaveis de ambientes obrigatórias
        * mapeando a porta de conexão do container (__8080__) com o host (__80__)
        * Também é feito o link com a rede
    
    
__Pronto!!! Conexão feita e ja pode ser usada, ao acessar o localhost:8080 no browser a aplicação rodará__

# Container LINKS

É um outro modo de comunicação entre os containers, porém não muito recomendado por ja estar no __modo legado__

* Método mais simples
    * Container de _origem_ tem acesso aos dados do container de _destino_ (__Não vice-versa__)
    * Permite conexão entre os containers pelos seus nomes
    
    
* Obsoleto - novas versões dockers podem não mais suportar
    * Muitos tutoriais ainda usam esse metodo

## Exemplo:

* Criando um container bosybox:
   * __docker run -itd --name l1 busybox__


* Criando outro container e linkando ele ao container __l1:__
    * __docker run -itd --name l2 --link l1:linkalias1 busybox__
        * __--link__ faz o link entre os containers, __l1:__ é o container destino (a origem é o l2) e __linkalias__ é apenas um apelido para o _destino_ e pode ser qquer coisa
        * O link é feito do __l2__ para o __l1__, o contrario não vale
            * __docker exec -it l2 ping l1 = True__
            * __docker exec -it l1 ping l2 = False__

### Case 

Fazendo uso do mesmo case apresendo anteriormente, envolvendo um banco de uma aplicação wordpress, podemos fazer o mesmo com o método legado de link

* Criando o container de banco:
    * __docker run -d --name db1 -e MYSQL_ROOT_PASSWORD=mwsql mysql:5.7__
    
    
* Criando o container da aplicação:
    * __docker run -d --name wp1 --link db1:meudb -p 8090:80 wordpress__
        * Bem mais simples, fizemos uso apenas do __--link__ e fazendo uso de apenas uma variável de ambiente que no caso é o nome que demos ao banco
        * E funciona!
        * Porem não temos quase nenhum controle e a flexibilidade é quase zero

# Exercicios

## Tarefa

Diante do conhecimento adquirido até aqui, você é capaz, mesmo em cenário não propício para isso (ambiente local single-host), disponibilizar um serviço simples de balanceamento de carga (load balancer) entre duas aplicações web similares, a partir de containers interconectados por redes bridges definidas pelo usuário. <br>

Para garantir a natureza de microserviços, você deve criar um container para cada finalidade:

1 container para o load balancer com nome "lb" + 1 diretório mapeado para arquivo de configuração

1 container web com nome "web1" + 1 volume mapeado para arquivos de FRONT do site

1 container web com nome "web2" + 1 volume mapeado para arquivos de FRONT do site

Onde, somente o container "load balancer" terá suas conexões na porta web 80 liberada, externamente. O restante dos containers (web1 e web2), somente, terão acesso permitido a partir do container balanceador (vide imagem superior de exemplo).

Assim, para garantir as interconexões seguras entre os containers, você deverá criar, adicionalmente, 1 rede bridge definida pelo usuário, e criar os containers nela:

1 rede bridge com nome lb-web

INFORMAÇÕES IMPORTANTES

Imagem para uso de containers "web": httpd (consulte mais em https://hub.docker.com/_/httpd)

Imagem para uso de container "load balancer": haproxy:1.6 (consulte mais em https://hub.docker.com/_/haproxy)

Diretório no container de configuração do balanceador: /usr/local/etc/haproxy/

Arquivo de configuração do balanceador: haproxy.cfg (em anexo)

Arquivos de FRONT para container web1: em anexo (fonte modificada em https://startbootstrap.com/themes/landing-page/)

Arquivos de FRONT para container web2: em anexo (fonte modificada em https://startbootstrap.com/themes/landing-page/)



### Qual comando usado para criar a rede bridge definida pelo usuário chamada "lb-web"?

__docker network create lb-web__

Por deafult a rede criada já é do driver bridge

### Quais comandos usados para criar os volumes vol-web1 e vol-web2; e os containers, em background, web1 e web2, ingressantes na rede criada, respectivamente?

__docker volume create vol-web1__

__docker volume create vol-web2__

__docker run -d --name web1 --net lb-web -v vol-web1:/usr/local/apache2/htdocs -p 9090:80 httpd__

__docker run -d --name web1 --net lb-web -v vol-web2:/usr/local/apache2/htdocs -p 9091:80 httpd__

### Quais comandos usados para enviar os arquivos, contidos em anexo, do FRONT dos websites dos containers web1 e web2, respectivamente?

Na dentro da pasta originals, que contem os arquivos q foram fornecidos:

__docker cp FRONT_web1/. web1:/usr/local/apache2/htdocs__

__docker cp FRONT_web2/. web2:/usr/local/apache2/htdocs__

### Qual comando usado para criar e iniciar o container balanceador, de nome "lb", disponibilizado na porta 80, ingressante da rede criada e com mapeamento de configuração no diretório /usr/local/etc/haproxy/ do container?

Mapeando a o diretório com que se encontra a rotina de load com o diretório onde persistem os dados da imagem hatproxy

__docker run -id --name lb --net lb-web -p 8080:80 -v /home/leandro/Desktop/original:/usr/local/etc/haproxy/ haproxy:1.6__